In [2]:
import networkx as nx
import random as rng
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
# import mathjax as mjax

In [14]:
class Colony:

    ants = []

    def __init__(self, ant_num, node_num, position=0):
        
        self.ants = []

        for i in range(0,ant_num):
            self.ants.append(Ant(position))
            self.ants[i].path = []


        self.unvisited_nodes = list(range(1,n)) # this assumes origin is 0

    
    def move(self,weighted_edges):
        
        ants = self.ants

        num_traveling_ants = 0

        diff = 0

        for i in ants:
            if i.pos != 0:
                num_traveling_ants += 1

        if (num_traveling_ants >= len(self.unvisited_nodes) and origin not in self.unvisited_nodes):
            print(list(i.pos !=0 for i in ants))
            diff = len(ants) - len(self.unvisited_nodes)
            self.unvisited_nodes.append(origin)


        for i in ants:
            i.unvisited_nodes = self.unvisited_nodes
            i.move(weighted_edges)
            if (i.pos != 0):
                self.unvisited_nodes.remove(i.pos)

            elif (i.pos == 0):
                if (diff == 0):
                    self.unvisited_nodes.remove(origin)
                else:
                    diff -= 1

In [4]:
class Ant:
    
    def __init__(self,position):
        self.pos = position

    path = []

    unvisited_nodes = []

    def edge_conditions(self,edge):
        
        if (edge[0] != self.pos and edge[1] != self.pos):
            return False

        # potential destination coordinate of ant from edge
        dest = edge[0] if edge[0] != self.pos else edge[1]

        if dest not in self.unvisited_nodes:
            return False
        
        elif (dest != 0 and edge in self.path):
            return False
        
        elif (self.path != []):

            if dest != origin:
                for i in self.path:
                    if (dest in i):
                        return False
                
        return True

    def move(self, weighted_edges):
        paths = list(filter(self.edge_conditions, weighted_edges.keys()))
        path_weights = list(weighted_edges[i] for i in paths)
        path = rng.choices(paths, path_weights)[0]
        self.pos = path[0] if path[0] != self.pos else path[1]
        self.path.append(path)

In [5]:
max_coord_size = 100 # maximum coordinate size

In [6]:
N = widgets.IntSlider(min=2,max=100,step=1, value=8, description = "n (number of nodes)")
A = widgets.IntSlider(min=1,max=100,step=1, value=3, description = "a (number of ants)")
M = widgets.FloatSlider(min=0.0,max=100.0,step=1, value=10, description = "K (pheremone)")
U = widgets.FloatSlider(min=0.0,max=1.0,step=0.01, value=0.5, description = "V (evaporation rate")

In [7]:
n=N.value
g = nx.complete_graph(n)

coords = {}

for i in g.nodes():
    coords[i] = (rng.uniform(0,max_coord_size),rng.uniform(0,max_coord_size))
    coords[0] = (0,0)

origin = 0 # MUST BE 0, needs work
    
weighted_edges = {}

for i in g.edges():
    weighted_edges[i] = 1

distance_edges = {}

for i in g.edges():
    distance_edges[i] = np.sqrt(((coords[i[1]])[0]-coords[i[0]][0])**2 + ((coords[i[1]])[1]-coords[i[0]][1])**2)

def dist(arr):
    dist = 0
    for i in arr:
        dist += distance_edges[i]
    
    return dist

In [16]:
V=U.value
K = M.value
# resets weights
for i in g.edges():
    weighted_edges[i] = 1

ant_num = A.value # number of ants per iteration
iteration_count = 1 # number of iterations

fitness = 1e10

colony = Colony(ant_num, n, origin)

for i in range(0,iteration_count):
    
    print("Iteration " + str(i))

    solution_found = False

    loop_counter = 0

    colony.__init__(ant_num, n, origin)

    for a in range(1,len(colony.ants)+1):
        print("Ant " + str(a) + " path so far:" +  str(colony.ants[a-1].path))

    while (solution_found == False and loop_counter < 10):
        
        while (colony.unvisited_nodes != [] and colony.ants[i].pos != 0 for i in range(0,len(colony.ants))):
            colony.unvisited_nodes
            print(colony.move(weighted_edges))
            for a in range(1,len(colony.ants)+1):
                print("Ant " + str(a) + " path so far:" +  str(colony.ants[a-1].path))
            

        total_distance = 0

        for a in colony.ants:
            for j in a.path:
                total_distance += distance_edges[j]

        if total_distance < fitness:
            fitness = total_distance
            solution_found = True
                
            for a in colony.ants:
                for j in a.path: # add pheremones from ants
                    weighted_edges[j] += K/distance_edges[j]
            
            loop_counter = 0

        for j in weighted_edges: # evaporate pheremones
            weighted_edges[j] = (1-V)*weighted_edges[j]
            if weighted_edges[j] == 0:
                weighted_edges[j] = 1e-12

        loop_counter += 1


    print("(Iteration " + str(i) + ") Total distance traveled: " + str(total_distance))


Iteration 0
Ant 1 path so far:[]
Ant 2 path so far:[]
Ant 3 path so far:[]
None
Ant 1 path so far:[(0, 5)]
Ant 2 path so far:[(0, 6)]
Ant 3 path so far:[(0, 7)]
None
Ant 1 path so far:[(0, 5), (1, 5)]
Ant 2 path so far:[(0, 6), (3, 6)]
Ant 3 path so far:[(0, 7), (4, 7)]
[True, True, True]
None
Ant 1 path so far:[(0, 5), (1, 5), (0, 1)]
Ant 2 path so far:[(0, 6), (3, 6), (2, 3)]
Ant 3 path so far:[(0, 7), (4, 7), (0, 4)]


IndexError: list index out of range

In [ ]:
for i in g.edges:
    g.add_edge(i[0],i[1],weight = weighted_edges[i])

g_christofides = nx.approximation.christofides(g)

g_christofides_edges = []
for i in range(0,len(g_christofides)-1):
    
    if g_christofides[i] > g_christofides[i+1]:
        g_christofides_edges.append((g_christofides[i+1],g_christofides[i]))

    else:
        g_christofides_edges.append((g_christofides[i],g_christofides[i+1]))

christofides_distance = sum(distance_edges[i] for i in g_christofides_edges)

g_sa = nx.approximation.simulated_annealing_tsp(g, nx.approximation.greedy_tsp(g))

g_sa_edges = []
for i in range(0,len(g_sa)-1):
    
    if g_sa[i] > g_sa[i+1]:
        g_sa_edges.append((g_sa[i+1],g_sa[i]))

    else:
        g_sa_edges.append((g_sa[i],g_sa[i+1]))

sa_distance = sum(distance_edges[i] for i in g_sa_edges)

max_weight = max(list(weighted_edges.values()))
min_weight = min(list(weighted_edges.values()))
total_distance = sum(distance_edges[i] for i in paths[0])

print("total distance traveled: " + str(total_distance))
print("Christofides algorithm distance traveled: " + str(christofides_distance))
print("Simulated Annealing algorithm distance traveled: " + str(sa_distance))
print("min pheremone: " + str(min_weight))
print("max pheremone: " + str(max_weight))

x =[]
for i in coords.values():
    x.append(i[0])


y =[]
for i in coords.values():
    y.append(i[1])

plt.scatter(x,y, color='black')
plt.scatter(coords[origin][0],coords[origin][1], color='green')

for i in range(0,len(x)):
    plt.text(x[i] * (1 + 0.005), y[i] * (1 + 0.01) , i, fontsize=12)


for i in paths[0]:
    plt.plot([coords[i[0]][0],coords[i[1]][0]], [coords[i[0]][1], coords[i[1]][1]], color='orangered')

# for i in g_christofides_edges:
#     plt.plot([coords[i[0]][0],coords[i[1]][0]], [coords[i[0]][1], coords[i[1]][1]], color='c')

# for i in g_sa_edges:
#     plt.plot([coords[i[0]][0],coords[i[1]][0]], [coords[i[0]][1], coords[i[1]][1]], color='r')

display(N)
display(M)
display(U)

NameError: name 'paths' is not defined

: 

: 

: 